<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/4%20Aprendizaje%20reforzado/Sesion%202/dqn_alm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEEP Q_LEARNING: DQN

In [1]:
import random
import gym
import numpy as np
from collections import deque
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model 

In [7]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def store(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state))

    def action(self, state):
        if random.randrange(self.action_size):
            return random.randrange(self.action_size)

    def experience_replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [8]:
env = gym.make('Acrobot-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32
EPISODES=20
rewards = []
for e in range(EPISODES):
    print(e)
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(100):
        print(time)
        # env.render()
        action = agent.action(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        rewards.append(reward)
        next_state = np.reshape(next_state, [1, state_size])
        agent.store(state, action, reward, next_state, done)
        state = next_state
       #if done:
       #    print("episode: {}/{}, score: {}, e: {:.2}"
       #          .format(e, EPISODES, time, agent.epsilon))
       #    break
        if len(agent.memory) > batch_size:
            agent.experience_replay(batch_size)

0
0
1
2
3
4


TypeError: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rewards)